<a href="https://colab.research.google.com/github/archiechang/study/blob/master/Kaggle/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# データの読み込み

In [2]:
#学習、訓練データを読み込み
train=pd.read_csv("https://raw.githubusercontent.com/archiechang/study/master/Kaggle/input/titanic/train.csv")
test=pd.read_csv("https://raw.githubusercontent.com/archiechang/study/master/Kaggle/input/titanic/test.csv")

In [3]:
#学習データを特徴量と目的変数に分ける
train_x=train.drop(["Survived"],axis=1)
train_y=train["Survived"]
test_x=test.copy()
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 特徴量の作成

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
#変数PassengerId,Name,Ticket,Cabinを除外する
train_x=train_x.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
test_x=test_x.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)

#それぞれのカテゴリ変数にlabel encodingを適用する
for c in["Sex","Embarked"]:
    le=LabelEncoder()
    #学習データ、テストデータを変換する
    train_x[c]=le.fit_transform(train_x[c].fillna("NA"))
    test_x[c]=le.fit_transform(test_x[c].fillna("NA"))
train_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,3
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,3
3,1,0,35.0,1,0,53.1000,3
4,3,1,35.0,0,0,8.0500,3


# モデル構築

In [9]:
from xgboost import XGBClassifier
import numpy as np

In [10]:
# モデルの作成及び学習データを与えての学習
# n_estimators決定木の数量
model=XGBClassifier(n_estimators=20,random_state=71)
model.fit(train_x,train_y)

#テストデータの予測値を確率で出力する
pred=model.predict_proba(test_x)[:,1]

#テストデータの予測値を二値に変換する
pred_label=np.where(pred>0.5,1,0)

#提出用ファイルの作成
submission=pd.DataFrame({"PassengerId":test["PassengerId"],"Survived":pred_label})
#submission.to_csv("submission.csv",index=False)


In [17]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# モデル評価

In [14]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold

In [18]:
#　各foldのスコアを保存するリスト
scores_accuracy=[]
scores_logloss=[]

#　クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in kf.split(train_x):
    #学習データを学習データとバリデーションデータに分ける
    tr_x,va_x=train_x.iloc[tr_idx],train_x.iloc[va_idx]
    tr_y,va_y=train_y.iloc[tr_idx],train_y.iloc[va_idx]

    #モデルの学習を行う
    model=XGBClassifier(n_estimators=20,random_state=71)
    model.fit(tr_x,tr_y)

    #バリデーションデータの予測値を確率で出力する
    va_pred=model.predict_proba(va_x)[:,1]

    #バリデーションデータでのスコアを計算する
    logloss=log_loss(va_y,va_pred)
    accuracy=accuracy_score(va_y,va_pred>0.5)

    #そのfoldのスコアを保存する
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

#各foldのスコアの平均を出力する
logloss=np.mean(scores_logloss)
accuracy=np.mean(scores_accuracy)
print(f"logloss: {logloss:.4f},accuracy: {accuracy:.4f}")


logloss: 0.4270,accuracy: 0.8148
